In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import sys
from datetime import datetime, timedelta
import pandas as pd
# import polars as pl
# import jellyfish
# import textdistance

from clickhouse_driver import Client
import pandas as pd
import numpy as np
import sys
import os
from datetime import datetime, timedelta

sys.path.append('../../utils')
from autodoc_connection import get_clickhouse_df


### tops

In [2]:
sql_top_30 = """
WITH

-- Parameters
toLastDayOfMonth(today()) AS curr_month
, toWeek(today()) - 1 AS curr_week
, 30 AS max_days
, (1) AS curr_tiers

-- Top 30% articles
, top_articles AS (
    SELECT
        country
        , article_id
        , if(max(end_date) = curr_month, 1, 0) AS in_last
    FROM core.top_revenue_article AS ta
    ARRAY JOIN arrayDistinct(arrayConcat(criteria1_country_code, criteria2_country_code)) AS country
    WHERE end_date BETWEEN toLastDayOfMonth(dateAdd(MONTH, -11, curr_month)) AND curr_month
    GROUP BY country, article_id
)

-- Competitors tiers
, comp_tiers AS (
    SELECT DISTINCT
        country_code AS country
        , competitor
        , tier
    FROM default.competitor_tier AS ct
    WHERE tier IN curr_tiers
)

-- Potential signals
, potential_signals AS (
    SELECT
        country
        , article_id
        , competitor
        , tier
        , in_last
    FROM top_articles AS ta
    LEFT JOIN comp_tiers AS ct
    ON ta.country = ct.country
)

-- Competitors data
, comp_signals AS (
    SELECT
        country_code AS country
        , article_id
        , company_name AS competitor
        , argMax(vendor, update_date) AS vendor
        , if(toWeek(min(partition_date)) < curr_week, 1, 0) AS is_old
        , if(toWeek(max(partition_date)) = curr_week, 1, 0) AS in_new
        , if(count(DISTINCT partition_date) > max_days / 7, 'daily', 'weekly') AS frequency
    FROM core_base.competitor_price_log AS cp
    INNER JOIN potential_signals AS ps
    ON cp.country_code = ps.country
        AND cp.article_id = ps.article_id
        AND cp.company_name = ps.competitor
    PREWHERE toYear(partition_date) = 2025
        AND toWeek(partition_date) <= curr_week
        AND competitor_tier IN curr_tiers
    GROUP BY country, article_id, competitor
)

-- Potential and competitors signals
, full_signals AS (
    SELECT
        country
        , toString(article_id) AS article_id
        , competitor
        , tier
        , in_last
        , if(cs.article_id > 0, 1, 0) AS is_scrapped
        , is_old
        , in_new
        , frequency
        , vendor
    FROM potential_signals AS ps
    GLOBAL LEFT JOIN comp_signals AS cs
    ON ps.country = cs.country
        AND ps.article_id = cs.article_id
        AND ps.competitor = cs.competitor 
)

-- Main
SELECT *, concat(country, article_id, competitor) AS key
FROM full_signals



FORMAT TabSeparatedWithNamesAndTypes
"""
print(f"Fetching top 30...")

df_30 = get_clickhouse_df(sql_top_30, 'mDWH')

Fetching top 30...


In [3]:
df_30

,country,article_id,competitor,tier,in_last,is_scrapped,is_old,in_new,frequency,vendor,key
0,ES,7064032,ebay.es,1,0,0,0,0,,,ES7064032ebay.es
1,ES,7064032,endado.com,1,0,0,0,0,,,ES7064032endado.com
2,ES,7064032,norauto.es,1,0,0,0,0,,,ES7064032norauto.es
3,ES,7064032,oscaro.es,1,0,1,1,1,daily,Bright Data,ES7064032oscaro.es
4,ES,7064032,feuvert.es,1,0,0,0,0,,,ES7064032feuvert.es
...,...,...,...,...,...,...,...,...,...,...,...
677790,ES,13552269,feuvert.es,1,0,0,0,0,,,ES13552269feuvert.es
677791,ES,13552269,amazon.es,1,0,0,0,0,,,ES13552269amazon.es
677792,CH,963644,amazon.ch,1,1,1,1,1,daily,Bright Data,CH963644amazon.ch
677793,CH,963644,ebay.ch,1,1,0,0,0,,,CH963644ebay.ch


In [4]:
df_30['article_id'] = df_30['article_id'].astype('Int64')

In [5]:
sql_top_50 = """
WITH

-- Parameters
toYear(today()) AS curr_year
, toMonth(today()) - 1 AS curr_month
, toWeek(today()) AS curr_week
, 30 AS max_days
, (1) AS curr_tiers


-- Top 30% articles
, top_articles AS (
    SELECT
        country_code as country
        , article_id
        , if(max(for_valid_month) = curr_month, 1, 0) AS in_last
    FROM dev_analytics_pdc.top_selection_50 AS ta
    WHERE for_valid_month <= curr_month
    GROUP BY country, article_id
)

-- Competitors tiers
, comp_tiers AS (
    SELECT DISTINCT
        country_code AS country
        , competitor
        , tier
    FROM default.competitor_tier AS ct
    WHERE tier IN curr_tiers
)

-- Potential signals
, potential_signals AS (
    SELECT
        country
        , article_id
        , competitor
        , tier
        , in_last
    FROM top_articles AS ta
    LEFT JOIN comp_tiers AS ct
    ON ta.country = ct.country
)

-- Competitors data
, comp_signals AS (
    SELECT
        country_code AS country
        , article_id
        , company_name AS competitor
        , argMax(vendor, update_date) AS vendor
        , if(toWeek(min(partition_date)) < curr_week - 1, 1, 0) AS is_old
        , if(toWeek(max(partition_date)) = curr_week - 1, 1, 0) AS in_new
        , if(count(DISTINCT partition_date) > max_days / 7, 'daily', 'weekly') AS frequency
    FROM core_base.competitor_price_log AS cp
    INNER JOIN potential_signals AS ps
    ON cp.country_code = ps.country
        AND cp.article_id = ps.article_id
        AND cp.company_name = ps.competitor
    PREWHERE toYear(partition_date) = curr_year
        AND toWeek(partition_date) < curr_week
        AND competitor_tier IN curr_tiers
    GROUP BY country, article_id, competitor
)

-- Potential and competitors signals
, full_signals AS (
    SELECT
        country
        , toString(article_id) AS article_id
        , competitor
        , tier
        , in_last
        , if(cs.article_id > 0, 1, 0) AS is_scrapped
        , is_old
        , in_new
        , frequency
        , vendor
    FROM potential_signals AS ps
    GLOBAL LEFT JOIN comp_signals AS cs
    ON ps.country = cs.country
        AND ps.article_id = cs.article_id
        AND ps.competitor = cs.competitor 
)

-- Main
SELECT *, concat(country, article_id, competitor) AS key
FROM full_signals

FORMAT TabSeparatedWithNamesAndTypes
"""
print(f"Fetching top 50...")

df_50 = get_clickhouse_df(sql_top_50, 'mDWH')
df_50

Fetching top 50...


,country,article_id,competitor,tier,in_last,is_scrapped,is_old,in_new,frequency,vendor,key
0,IT,7691317,amazon.it,1,1,1,1,1,daily,Bright Data,IT7691317amazon.it
1,IT,7691317,ricambi-smc.it,1,1,0,0,0,,,IT7691317ricambi-smc.it
2,IT,7691317,ebay.it,1,1,1,1,1,daily,Bright Data,IT7691317ebay.it
3,IT,7691317,norauto.it,1,1,0,0,0,,,IT7691317norauto.it
4,IT,7691317,mister-auto.it,1,1,1,1,1,daily,Competera,IT7691317mister-auto.it
...,...,...,...,...,...,...,...,...,...,...,...
1482037,IT,7624220,amazon.it,1,1,1,1,1,daily,Bright Data,IT7624220amazon.it
1482038,IT,7624220,ricambi-smc.it,1,1,1,1,0,daily,Bright Data,IT7624220ricambi-smc.it
1482039,IT,7624220,ebay.it,1,1,1,1,1,daily,Competera,IT7624220ebay.it
1482040,IT,7624220,norauto.it,1,1,0,0,0,,,IT7624220norauto.it


In [6]:
df_50['article_id'] = df_50['article_id'].astype('Int64')

In [7]:


def get_segmented_sales_data(ref_date_str: str = '2025-06-01'):
    

    # --- Define Parameters for the SQL Query ---
    strategic_categories = ['Brake Discs', 'Shock Absorbers Components', 'Lubrication Products', 'Engine Oil Products', 'Oil Additives']
    specific_generic_ids = [
    1862, 479, 3096, 259, 391, 3902, 3437, 50, 1256, 429, 542, 4921, 189, 307,
    100215, 2905, 3859, 190, 3675, 3436, 560, 198, 443, 1263, 3901, 7413, 395,
    1191, 637, 262, 2021, 4172, 324, 937, 3175, 567, 607, 1387, 1431, 3702,
    3884, 261]


    sql_query = f"""
    WITH
        toDate('{ref_date_str}') AS ref_date,
        {strategic_categories} AS strategic_categories_list,
        {specific_generic_ids} AS specific_generic_ids_list,

    base_sales AS (
        SELECT
            multiIf(upper(origin) IN ('BF', 'BN'), 'BE', upper(origin) IN ('LF', 'LD'), 'LU', upper(origin) IN ('LR'), 'LV', upper(origin) IN ('EN'), 'GB', upper(origin) IN ('CF'), 'CH', upper(origin) IN ('DG'), 'DE', upper(origin)) AS country,
            article_id,
            dictGet('default.tecdoc_article_src', 'generic_id', toUInt64(article_id)) AS generic_id,
            dictGetOrDefault('default.category_tree_reporting', 'subcategory', toUInt64(generic_id), 'W/O Category') AS subcategory,
            dictGetOrDefault('default.category_tree_reporting', 'main_category', toUInt64(generic_id), 'W/O Category') AS category,
            order_or_refund_created_date_berlin AS sale_date,
            money_total_product_navision AS revenue
        FROM analytics.product_sales_2
        WHERE
            sale_date >= date_sub(MONTH, 3, ref_date) AND sale_date < date_add(MONTH, 1, ref_date)
            AND operation = 'sales'
            AND country IN ('DE', 'FR', 'ES', 'IT', 'GB', 'SE', 'AT', 'BE', 'NL', 'CH')
            AND money_total_product_navision > 0
    ),
    category_asp AS (
        SELECT category, sum(revenue) / count() AS asp FROM base_sales GROUP BY category
    ),
    top_5_asp_categories AS (
        SELECT category FROM category_asp ORDER BY asp DESC LIMIT 5
    ),
    article_aggregated_flags AS (
        SELECT
            country, article_id, any(generic_id) AS generic_id, any(category) AS category,any(subcategory) AS subcategory,
            max(if(sale_date >= ref_date, 1, 0)) AS had_sales_t1m,
            max(if(sale_date >= date_sub(MONTH, 2, ref_date), 1, 0)) AS had_sales_t3m,
            1 AS had_sales_t4m
        FROM base_sales GROUP BY country, article_id
    )
    
    SELECT
        agg.country, agg.article_id,
        if(agg.category IN (SELECT category FROM top_5_asp_categories) AND agg.had_sales_t1m = 1, 1, 0) AS top_5_asp_cat_t1m,
        if(agg.category IN (SELECT category FROM top_5_asp_categories) AND agg.had_sales_t3m = 1, 1, 0) AS top_5_asp_cat_t3m,
        agg.had_sales_t1m AS has_sales_t1m,
        agg.had_sales_t3m AS has_sales_t3m,
        agg.had_sales_t4m AS has_sales_t4m,
        if(agg.subcategory IN strategic_categories_list AND agg.had_sales_t1m = 1, 1, 0) AS strategic_cat_t1m,
        if(agg.subcategory IN strategic_categories_list AND agg.had_sales_t3m = 1, 1, 0) AS strategic_cat_t3m,
        if(agg.generic_id IN specific_generic_ids_list AND agg.had_sales_t3m = 1, 1, 0) AS specific_generic_ids_t3m
    FROM article_aggregated_flags AS agg
    FORMAT TabSeparatedWithNamesAndTypes
    """

    # --- Execute the query and load data into a pandas DataFrame ---
    print("Executing segmentation query...")
    df = get_clickhouse_df(sql_query, 'bsn')
    return df

# --- How to use the function ---
if __name__ == "__main__":
    # Get the segmented data for June 2025
    segmented_df = get_segmented_sales_data(ref_date_str='2025-06-01')

    if segmented_df is not None:
        print("\n--- Sample of the Final Segmented Data ---")

        # You can now work with the DataFrame
        print("\n--- Summary of Segments ---")
        # Exclude country and article_id for the summary
        segment_columns = [col for col in segmented_df.columns if col not in ['country', 'article_id']]
        print(segmented_df[segment_columns].sum())

Executing segmentation query...

--- Sample of the Final Segmented Data ---

--- Summary of Segments ---
top_5_asp_cat_t1m             77761
top_5_asp_cat_t3m            160198
has_sales_t1m               1278928
has_sales_t3m               2505445
has_sales_t4m               2967514
strategic_cat_t1m            106889
strategic_cat_t3m            194939
specific_generic_ids_t3m     187660
dtype: int64


In [8]:
segmented_df.columns

Index(['country', 'article_id', 'top_5_asp_cat_t1m', 'top_5_asp_cat_t3m',
       'has_sales_t1m', 'has_sales_t3m', 'has_sales_t4m', 'strategic_cat_t1m',
       'strategic_cat_t3m', 'specific_generic_ids_t3m'],
      dtype='object')

In [9]:
duplicates = segmented_df[segmented_df.duplicated(subset=['country', 'article_id'], keep=False)]
duplicates

,country,article_id,top_5_asp_cat_t1m,top_5_asp_cat_t3m,has_sales_t1m,has_sales_t3m,has_sales_t4m,strategic_cat_t1m,strategic_cat_t3m,specific_generic_ids_t3m


In [10]:
sum_columns = [
    'top_5_asp_cat_t1m', 'top_5_asp_cat_t3m',
    'has_sales_t1m', 'has_sales_t3m', 'has_sales_t4m',
    'strategic_cat_t1m', 'strategic_cat_t3m',
    'specific_generic_ids_t3m'
]

# Group by country and sum selected columns
t1_by_country_sales = (
    segmented_df.groupby('country')[sum_columns]
    .sum()
    .reset_index()
)
t1_by_country_sales

,country,top_5_asp_cat_t1m,top_5_asp_cat_t3m,has_sales_t1m,has_sales_t3m,has_sales_t4m,strategic_cat_t1m,strategic_cat_t3m,specific_generic_ids_t3m
0,AT,3910,10061,58537,133623,166078,5958,12409,10345
1,BE,4699,10561,72735,154476,187607,6800,12947,12517
2,CH,2614,6259,59350,134702,166980,6031,12642,11345
3,DE,20786,40358,305262,552299,637592,23358,38645,44427
4,ES,6150,13061,127572,256331,305680,10296,19212,19083
5,FR,20903,37381,304687,536443,615481,21890,35927,42898
6,GB,3372,7366,95208,195766,236901,9968,18925,11153
7,IT,6814,15449,99072,208261,250160,8792,16727,13932
8,NL,4927,10891,80800,170923,207391,7083,13730,12357
9,SE,3586,8811,75705,162621,193644,6713,13775,9603


### comp data

In [11]:

sql_query_comp = f"""
    WITH

        toYear(today()) AS curr_year
, toMonth(today()) - 1 AS curr_month
, toWeek(today()) AS curr_week
, 30 AS max_days
, (1) AS curr_tiers


    SELECT
        country_code AS country
        , article_id
        , company_name AS competitor
        , argMax(vendor, update_date) AS vendor
        , 1 as is_scraped
    FROM core_base.competitor_price_log AS cp
    PREWHERE toYear(partition_date) = 2025
        AND toWeek(partition_date) <= curr_week
        AND competitor_tier IN curr_tiers
    GROUP BY country, article_id, competitor, is_scraped



FORMAT TabSeparatedWithNamesAndTypes
    """
df_comp = get_clickhouse_df(sql_query_comp, 'mDWH')


In [12]:
df_scrapped = df_comp

In [13]:

sql_query_tiers = f"""
    WITH

        toYear(today()) AS curr_year
, toMonth(today()) - 1 AS curr_month
, toWeek(today()) AS curr_week
, 30 AS max_days
, (1) AS curr_tiers


    -- Competitors tiers

    SELECT DISTINCT
        country_code AS country
        , competitor
        , tier
    FROM default.competitor_tier AS ct
    WHERE tier IN curr_tiers




FORMAT TabSeparatedWithNamesAndTypes
    """
df_comp_tiers = get_clickhouse_df(sql_query_tiers, 'mDWH')
df_comp_tiers

,country,competitor,tier
0,AT,kfzteile24.at,1
1,AT,atp-autoteile.at,1
2,AT,ebay.at,1
3,AT,amazon.at,1
4,AT,mister-auto.at,1
...,...,...,...
59,PT,norauto.pt,1
60,SE,amazon.se,1
61,SE,trodo.se,1
62,SE,skruvat.se,1


### table 2

In [14]:
# --- 1. Data Preparation: Combine ALL use cases into one long-format DataFrame ---

# a) Melt the base segments
id_vars = ['country', 'article_id']
use_case_columns = [col for col in segmented_df.columns if col not in id_vars]
long_segments_base = pd.melt(
    segmented_df,
    id_vars=id_vars,
    value_vars=use_case_columns,
    var_name='use_case',
    value_name='is_in_segment'
)
long_segments_base = long_segments_base[long_segments_base['is_in_segment'] == 1].drop(columns='is_in_segment')

# b) Process the new percentile DataFrames
def process_percentile_df(df, prefix):
    last_month_segments = df[df['in_last'] == 1][['country', 'article_id']].copy()
    last_month_segments['use_case'] = f'{prefix} Last Month'

    ttm_segments = df[df['in_last'].isin([0, 1])][['country', 'article_id']].copy()
    ttm_segments['use_case'] = f'{prefix} TTM'

    return pd.concat([last_month_segments, ttm_segments]).drop_duplicates().reset_index(drop=True)

top_30_segments = process_percentile_df(df_30, "Top30%")
top_50_segments = process_percentile_df(df_50, "Top50%")

# c) Concatenate all segments into one master list
long_segments = pd.concat([
    long_segments_base,
    top_30_segments,
    top_50_segments
]).drop_duplicates().reset_index(drop=True)

long_segments['article_id'] = long_segments['article_id'].astype('Int64')
long_segments

,country,article_id,use_case
0,FR,7070014,top_5_asp_cat_t1m
1,DE,15222750,top_5_asp_cat_t1m
2,DE,18755876,top_5_asp_cat_t1m
3,FR,23008920,top_5_asp_cat_t1m
4,IT,20030524,top_5_asp_cat_t1m
...,...,...,...
8083420,GB,13710233,Top50% TTM
8083421,SE,15236966,Top50% TTM
8083422,DE,19225233,Top50% TTM
8083423,NL,15048467,Top50% TTM


In [15]:

# d) Create the final master analysis grid
master_grid = pd.merge(long_segments, df_comp_tiers, on='country', how='inner')
master_grid = pd.merge(
    master_grid,
    df_scrapped,
    on=['country', 'article_id', 'competitor'],
    how='left'
)
master_grid['is_scraped'] = master_grid['is_scraped'].fillna(0).astype(int)



In [16]:
master_grid.use_case.unique()

array(['top_5_asp_cat_t1m', 'top_5_asp_cat_t3m', 'has_sales_t1m',
       'has_sales_t3m', 'has_sales_t4m', 'strategic_cat_t1m',
       'strategic_cat_t3m', 'specific_generic_ids_t3m',
       'Top30% Last Month', 'Top30% TTM', 'Top50% Last Month',
       'Top50% TTM'], dtype=object)

In [17]:

# --- 2. Build Table 1: Target SKUs by Country and Use Case ---
table1_pivot = pd.pivot_table(
    long_segments,
    values='article_id',
    index='country',
    columns='use_case',
    aggfunc='nunique',
    fill_value=0
)
deduped_skus_per_country = long_segments.groupby('country')['article_id'].nunique()
table1_pivot['deduped_skus'] = deduped_skus_per_country

print("--- TABLE 1: Target SKUs by Country ---")
table1_pivot



--- TABLE 1: Target SKUs by Country ---


use_case,Top30% Last Month,Top30% TTM,Top50% Last Month,Top50% TTM,has_sales_t1m,has_sales_t3m,has_sales_t4m,specific_generic_ids_t3m,strategic_cat_t1m,strategic_cat_t3m,top_5_asp_cat_t1m,top_5_asp_cat_t3m,deduped_skus
country,,,,,,,,,,,,,
AT,2103,13396,9996,23866,58537,133623,166078,10345,5958,12409,3910,10061,176758
BE,2073,12510,10404,24187,72735,154476,187607,12517,6800,12947,4699,10561,197436
CH,1897,12659,8828,23276,59350,134702,166980,11345,6031,12642,2614,6259,176927
DE,4953,17484,25536,46590,305262,552299,637592,44427,23358,38645,20786,40358,645172
ES,3595,18288,17233,38738,127572,256331,305680,19083,10296,19212,6150,13061,318364
FR,4566,14948,23150,42086,304687,536443,615481,42898,21890,35927,20903,37381,621299
GB,2577,13167,11930,26726,95208,195766,236901,11153,9968,18925,3372,7366,245955
IT,2504,13506,12831,27720,99072,208261,250160,13932,8792,16727,6814,15449,259698
NL,2022,11705,10376,23564,80800,170923,207391,12357,7083,13730,4927,10891,216113


In [18]:

# --- 3. Build Table 2: Assumed URLs by Competitor and Use Case ---
scraped_items = master_grid[master_grid['is_scraped'] == 1]
table2_pivot = pd.pivot_table(
    scraped_items,
    values='article_id',
    index='competitor',
    columns='use_case',
    aggfunc='nunique',
    fill_value=0
)
deduped_urls_per_competitor = scraped_items.groupby('competitor')['article_id'].nunique()
table2_pivot['deduped_assumed_urls'] = deduped_urls_per_competitor

print("--- TABLE 2: Assumed URLs by Competitor ---")
table2_pivot


--- TABLE 2: Assumed URLs by Competitor ---


use_case,Top30% Last Month,Top30% TTM,Top50% Last Month,Top50% TTM,has_sales_t1m,has_sales_t3m,has_sales_t4m,specific_generic_ids_t3m,strategic_cat_t1m,strategic_cat_t3m,top_5_asp_cat_t1m,top_5_asp_cat_t3m,deduped_assumed_urls
competitor,,,,,,,,,,,,,
amazon.at,1367,7223,6204,12713,26177,55062,66777,5272,2980,5611,1435,3391,71078
amazon.ch,782,4177,3713,6821,15164,27311,31315,2852,1337,2232,533,1028,33340
amazon.co.uk,1660,6819,6747,13005,31350,57421,66528,4380,3358,5461,845,1580,69594
amazon.com.be,1372,6429,5885,11708,21171,36731,41839,4289,2134,3316,1441,2851,45281
amazon.de,4165,12108,17941,27876,117616,178139,194147,19029,10632,15114,7600,12830,196370
amazon.es,2702,11978,11947,23867,52718,93976,107523,9765,5549,9232,2046,3829,113310
amazon.fr,3822,10772,16799,26484,123006,187552,205459,21180,10658,15521,7066,11077,207334
amazon.it,1927,8979,9013,16889,42644,79154,91104,7150,4688,7875,2790,5736,95482
amazon.nl,1450,6791,6589,12672,28816,51955,59862,4654,2878,4579,1684,3361,63484


In [19]:


# --- 4. Build Table 3: Assumed URLs by Vendor and Use Case ---
table3_pivot = pd.pivot_table(
    scraped_items,
    values='article_id',
    index='vendor',
    columns='use_case',
    aggfunc='nunique',
    fill_value=0
)
deduped_urls_per_vendor = scraped_items.groupby('vendor')['article_id'].nunique()
table3_pivot['deduped_sum'] = deduped_urls_per_vendor

print("--- TABLE 3: Assumed URLs by Vendor ---")
table3_pivot

--- TABLE 3: Assumed URLs by Vendor ---


use_case,Top30% Last Month,Top30% TTM,Top50% Last Month,Top50% TTM,has_sales_t1m,has_sales_t3m,has_sales_t4m,specific_generic_ids_t3m,strategic_cat_t1m,strategic_cat_t3m,top_5_asp_cat_t1m,top_5_asp_cat_t3m,deduped_sum
vendor,,,,,,,,,,,,,
Bright Data,11985,41165,47708,77212,208211,303520,330972,39700,19084,25677,14947,23030,338111
Competera,8947,28351,34225,54281,127893,187542,204684,24906,12853,18065,11307,19707,210562
Infatica,1430,11229,11806,22941,131417,205182,224886,9496,10847,17002,5925,9705,236445
